In [ ]:
# Testing Cell
from aviary.api import LegacyCode
from aviary.interface.methods_for_level2 import AviaryGroup
from aviary.utils.doctape import glue_variable, glue_class_options

current_glued_vars = []
glue_class_options(AviaryGroup, current_glued_vars, md_code=True)

glue_variable(LegacyCode.FLOPS.name, md_code=False)
glue_variable(LegacyCode.GASP.name, md_code=False)

# Features and Functionalities

## Subsystems

At a minimum, an Aviary model needs to include the following subsystems:

- Geometry
- Mass
- Aerodynamics
- Propulsion
- Mission

Basically, these subsystems provide all the necessary forces acting on the aircraft which allows us to evaluate the performance of the aircraft across its mission.
Aviary does not explicitly support models with only some of these subsystems, but you are welcome to use parts of Aviary's code to build your own model.

The following sections will discuss each of these subsystems in more detail.

## Basic Assumptions

Because Aviary is a tool for conceptual design, we make a number of assumptions about the aircraft and mission that allow us to simplify the model.
These are often baked in to the underlying subsystem models, but it's important to understand what these assumptions are so that you can make sure they're appropriate for your problem.

For example, Aviary's computed aerodynamic models are only valid for subsonic flight.
Aviary itself is not limited to subsonic flight, but the aerodynamic models are.
So, you could use Aviary to model a supersonic aircraft, but you would need to provide your own aerodynamic model.

Other assumptions are largely dependent on which subsystems you include, the aircraft you're designing, and the mission you're evaluating.

<!-- TODO: add more detail here -->

## Mission Optimization

We'll now discuss mission optimization from a user's perspective.
What we mean by that is that we'll not go into details on the theory or math behind mission optimization, but instead focus on what it means for you to set up a mission optimization problem in Aviary.
We'll start by discussing the different phases of a mission that you can model in Aviary, and then we'll discuss how to set up constraints for your mission optimization problem.
Additionally, we'll dig into just how much flexibility you should give the optimizer when defining your mission.

### Basic definition of a mission

Throughout Aviary we use a series of terms when discussing mission optimization.

A ["trajectory"](https://openmdao.github.io/dymos/features/phases/phases.html) is the full mission that the aircraft flies.
Usually this is from takeoff to landing, inclusive.
Sometimes you might want to model just a portion of the full aircraft trajectory; for example only the cruise portion.

A ["phase"](https://openmdao.github.io/dymos/features/phases/phases.html) is a part of the trajectory that is defined by a single set of differential equations.
For example, a simple way of defining a full trajectory is to have climb, cruise, and descent phases.
Each of these phases can have different physics, subsystems, controls, and constraints.
For example, the climb phase for a hybrid-electric aircraft might be have electric motor assistance whereas that might not be needed for the cruise phase.

A "segment" is a sub-portion of a phase that is mostly used internally or when discussing the math behind the problem.
Users that are not defining custom phases will likely never need to worry about segments.

### Defining a mission

A mission is defined by a series of phases that the user chooses by specifying options in the {glue:md}`mission_info` dictionary.
The {glue:md}`mission_info` dictionary is a dictionary of dictionaries, where each key is the name of a phase and the value is a dictionary of options for that phase.

How you choose to define your phases is dependent on the aircraft you're modeling, the mission you're trying to evaluate, and the flexibility you want to give the optimizer.
For example, if you have a relatively conventional aircraft that is flying a straightforward mission, you might just need three phases: climb, cruise, and descent.
However, if you have a more complex aircraft or mission, you might need to define more phases.
For instance, if you're modeling a hybrid-electric aircraft with non-conventional propulsion systems that are controlled in different ways, you might want to define additional phases and prescribe different options based on which physics you want included at different stages in the flight.

In general, if you're familiar with the legacy tools {glue:md}`FLOPS` or {glue:md}`GASP`, you can use the corresponding default {glue:md}`mission_info` objects to start defining your mission.
FLOPS-based missions have three integrated phases: climb, cruise, and descent, as well as analytic takeoff and landing systems.
GASP-based missions have at least nine integrated phases: {glue:md}`groundroll`, {glue:md}`rotation`, {glue:md}`ascent`, {glue:md}`accel`, {glue:md}`climb1`, {glue:md}`climb2`, {glue:md}`cruise`, {glue:md}`desc1`, and {glue:md}`desc2`, as well landing systems.
The difference in the number of phases is due to the fact that {glue:md}`GASP` had more detailed requirements on the flight profile, especially in the early phases of a mission.

In [ ]:
# Testing Cell
from aviary.utils.doctape import check_value, get_variable_name, glue_variable

expected_keys = [
    'groundroll',
    'rotation',
    'ascent',
    'accel',
    'climb1',
    'climb2',
    'cruise',
    'desc1',
    'desc2',
]
for key in expected_keys:
    glue_variable(key, md_code=False)

You can import a copy of the default {glue:md}`mission_info` dicts and then modify them as you need to for your own mission definition.

### Defining mission controls and constraints

How you choose to define your mission constraints depends on the aircraft your modeling, the equations of motion used, and which subsystems you're including in your model.
For example, if you're modeling a single-aisle commercial transport aircraft that will fly a relatively conventional mission, you might define your mission so that the aircraft can only climb in the first phase, cruises at a fixed altitude and Mach number, then descends in the final phase.
This would mimic the actual flight profile of this aircraft to a reasonable degree.

However, if you're modeling an urban air mobility aircraft that will fly a more complex mission, you might want to give the optimizer more flexibility in how it flies the mission.
Purposefully giving the optimizer the freedom to explore the trajectory design space at the same time it's designing the aircraft is a perfect example use case for Aviary.
This will result in a more complex optimization problem that might not converge well without some expert knowledge of the problem, but it will allow you to explore the design space more fully.

